https://github.com/apoorvam/smart-sensor

# IMPORTING LIBRARIES

In [14]:
import os
from kinopy.io import read_record
from scipy.signal import detrend, find_peaks, medfilt
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error
import neurokit2 as nk
from psd2 import psd2
import pylops
import time
import pandas as pd 
import scipy as sp
import math
np.random.seed(1)

# DATA LOADING

In [2]:
main_path = "/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/"
path_all = []
for i in sorted(os.listdir(main_path)):
    sub_path = os.path.join(main_path, i)
    sub = sorted(os.listdir(sub_path))
    for k in sub:
        subsub_path = os.path.join(sub_path, k)
        subsub = sorted(os.listdir(subsub_path))
        if len(subsub) == 2:
            path_all.append((os.path.join(subsub_path, str(subsub[0]))))
            path_all.append((os.path.join(subsub_path, str(subsub[1]))))
        else:
            path_all.append((os.path.join(subsub_path, str(subsub[0]))))
            
path_all

['/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/BDC10/K-ICB_A_BDC10_20190330-101553_gwoB4aykkTA8fy',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/BDC9/K-ICB_A_BDC9_20190331-111511_BKe5wNpmMCVETk',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/BDC9/K-ICB_A_BDC9_20190331-112912_BKe5wrlWOhVETk',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/FAM/K-ICB_A_FAM_20190330-100233_vk0azV9DkUKXi4',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/HDT21/K-ICB_A_HDT21_20190429-110822_gwoxGB2nzSA8fy',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/HDT5/K-ICB_A_HDT5_20190413-090257_BKe0M07rPhVETk',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/HDT56/K-ICB_A_HDT56_20190603-110736_gwokQXN5vFA8fy',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/R1/K-ICB_A_R1_20190609-131324_bB5V5zalQtYqid',
 '/home/giuseppeditursi/Documents/Data/K-ICB/raw_v2/Sub-A/R1/K-ICB_A_R1_20190609-132625_bB5V5pwQQSYqid',
 '/home/giuseppeditursi

In [3]:
measurement = path_all[1]
record = read_record(measurement)
record

<Record 'K-ICB_A_BDC9_20190331-111511_BKe5wNpmMCVETk' (signals: 3, segmentations: 0, metrics: 0)>
Meta Data:
    id                      K-ICB_A_BDC9_20190331-111511_BKe5wNpmMCVETk
    code                    BDC9
    date                    2019-03-31T11:15:11+00:00
    subject                 A | Male | 1.8m | 72.8kg
    phone                   iPhone | iOS N/A | sw:2018.0.3.2
    device                  1.06 | hw:2017.0 | fw:2018.0.4.0
Signals:
    'ecg'                   electric/ecg        (451.06s, 200Hz)
    'scg'                   mechanical/scg      (451.06s, 200Hz)
    'bcg'                   mechanical/bcg      (451.06s, 200Hz)

In [4]:
fs = 200

In [5]:
scg = record.signals["scg"]
scg

<Biosignal 'scg' mechanical/scg (451.06s, 200Hz)>
Subsignals:  dis, vel, acc
Dimensions:  (motion: 2, time: 90212, axis: 3)

In [6]:
x = 0
y = 1
z = 2

# Lin
typ = 0
s1 = detrend(scg.vel[typ][:,x])      
s2 = detrend(scg.vel[typ][:,y])      
s3 = detrend(scg.vel[typ][:,z])      

# Rot
typ = 1
s4 = detrend(scg.vel[typ][:,x])       
s5 = detrend(scg.vel[typ][:,y])        
s6 = detrend(scg.vel[typ][:,z])  

# DATA PROCESSING

## RESPIRATION EXTRACTION

Apply any algorithm for respiration extraction over the three axis and merge them in a matrix.

In [10]:
s = [s4, s5, s6]

resp_all = []

for idx, i in enumerate(s):
    print(idx)
    
    #... respiration extraction algorithm
    
    resp = detrend(resp)
    resp_all.append(resp)

0
1
2


In [11]:
resp_x = np.vstack(resp_all[0])
resp_y = np.vstack(resp_all[1])
resp_z = np.vstack(resp_all[2])
resp = np.squeeze(np.stack((resp_x, resp_y, resp_z), axis=-1))

## RESPIRATION PERIOD COMPUTATION

### Kalman Filter

Rather than just using a average of estimates from each axis, we use a multi-axis fusion approach to improve estimation accuracy.
To fuse the estimates from X, Y, Z axes a Kalman Filter is implemented. Historical repiratory rates are used for prediction and current estimate is used as measurement for model.

In [21]:
def get_segment(data, segment_number, segment_size):
  return data[segment_number*segment_size: segment_number*segment_size+segment_size]

def fft(data):
  data = data - data.mean()
  N = len(data)
  data = sp.signal.detrend(data)  
  fft_data = sp.fftpack.fft(data)
  f = np.linspace(0, N/fs, N)

  max_amp = 0
  max_index = 0
  index = 0
  for c in fft_data[:N // 2]:
    if f[index] < 0.13 or f[index] > 0.5:
      index = index + 1
      continue;

    real = np.real(c)
    img = np.imag(c)
    amp = np.sqrt(real*real + img*img)
    if max_amp < amp:
      max_amp = amp
      max_index = index
    index = index+1
  return f, fft_data, max_amp, max_index

# Apply Kalman filter for multi axis data fusion
def apply_kalman_filter(data):
  print("Performing multi-axis fusion by Kalman filter...")
  segment_size_for_kalman = 4 * fs
  n_segments = math.ceil(len(data)/segment_size_for_kalman) +1
  rhat = np.zeros(n_segments)    # a posteri estimate of rr
  rhatminus = np.zeros(n_segments) # a priori estimate of rr
  p = np.zeros(n_segments)  # a posteri error estimate
  pminus = np.zeros(n_segments)  # a priori error estimate
  k = np.zeros(n_segments) # kalman gain

  r_measurement = np.zeros((n_segments, 3)) # observation measurements

  # initial guesses
  p[0] = 1.0
  rhat[0] = 30
  for segment_number in range(1, n_segments):
    segment = get_segment(data, segment_number-1, segment_size_for_kalman)
    f_x, fft_data_x, max_amp_x, max_index_x = fft(segment[:,0])
    f_y, fft_data_y, max_amp_y, max_index_y = fft(segment[:,1])
    f_z, fft_data_z, max_amp_z, max_index_z = fft(segment[:,2])
    r_measurement[segment_number] = [f_x[max_index_x] * 60, f_y[max_index_y] * 60, f_z[max_index_z] * 60]  
    variance_x = np.var(segment[:,0])
    variance_y = np.var(segment[:,1])
    variance_z = np.var(segment[:,2])
    total_variance = variance_x + variance_y + variance_z

    # time update
    rhatminus[segment_number] = rhat[segment_number-1]
    pminus[segment_number] = p[segment_number-1]

    # measurement update
    n_f = [variance_x/total_variance, variance_y/total_variance, variance_z/total_variance]
    z = (n_f[0] * r_measurement[segment_number][0]) + (n_f[1] * r_measurement[segment_number][1]) + (n_f[2] * r_measurement[segment_number][2])
    r = np.sqrt(np.sum(list(map(lambda index: np.square(n_f[index] * (r_measurement[segment_number][index] - z)), [0, 1, 2])))) # variance of the measurement noise
    if (pminus[segment_number]!=0 or r != 0):
      k[segment_number] = pminus[segment_number] /(pminus[segment_number] + r)

    rhat[segment_number] = rhatminus[segment_number] + k[segment_number] * (z - rhatminus[segment_number]) 
    p[segment_number] = (1 - k[segment_number]) * pminus[segment_number]

  print("Breathing rate from Kalman filter:", rhat[n_segments-1])
  return rhat[n_segments-1]

In [22]:
br = apply_kalman_filter(resp)

Performing multi-axis fusion by Kalman filter...
Breathing rate from Kalman filter: 7.809762202753442
